## Importing necessary packages and data

In [1]:
import numpy as np
import pandas as pd
from datetime import date

## 1. Forward Pricing Fundamentals

### Data

Consider a Treasury note with the following characteristics:

| Field | Value |
|-------|-------|
| Coupon | 4.00% |
| Maturity | 2030-02-28 |
| Issue Date | 2023-02-28 |
| First Coupon | 2023-08-31 |

Market conditions as of **2023-04-18**:

| Field | Value |
|-------|-------|
| Spot Price | 102-02 (i.e., 102 + 2/32) |
| Repo Rate | 4.85% |
| Forward Date | 2023-08-01 |

### 1.1 Accrued Interest

To calculate the accrued interest $A_t$ and $A_T$, we must adhere to the Actual/Actual day count convention used for US Treasuries.

**Key Dates:**
- **Current Date ($t$):** 2023-04-18
- **Forward Date ($T$):** 2023-08-01

**Reference Period:**  
The coupon period runs from the Issue Date (**2023-02-28**) to the First Coupon Date (**2023-08-31**).

**Formula for accrued interest:**  
$$
A = \frac{C * N}{2} \times \frac{\text{Days since last coupon}}{\text{Days in current coupon period}}
$$

In [2]:
# Inputs
face_value = 100
coupon_rate = 0.04
issue_date = date(2023, 2, 28)
first_coupon_date = date(2023, 8, 31)
current_date = date(2023, 4, 18)  # t
forward_date = date(2023, 8, 1)   # T

# Helper functions for day counts
def days_between(d1, d2):
    return (d2 - d1).days

# Accrued Interest Calculation
# Coupon period days
days_in_period = days_between(issue_date, first_coupon_date)

# Accrued at t (Current Date)
days_accrued_t = days_between(issue_date, current_date)
accrued_t = ((coupon_rate * face_value) / 2) * (days_accrued_t / days_in_period)

# Accrued at T (Forward Date)
days_accrued_T = days_between(issue_date, forward_date)
accrued_T = ((coupon_rate * face_value) / 2) * (days_accrued_T / days_in_period)

# Display the accrued interest data
accrued_data = {
    "Description": [
        "Days in coupon period (Feb 28 - Aug 31)",
        "Days accrued at t (Feb 28 - Apr 18)",
        "Accrued Interest at t (A_t)",
        "Days accrued at T (Feb 28 - Aug 01)",
        "Accrued Interest at T (A_T)"
    ],
    "Value": [
        days_in_period,
        days_accrued_t,
        f"{accrued_t:.6f}",
        days_accrued_T,
        f"{accrued_T:.6f}"
    ]
}

df_accrued = pd.DataFrame(accrued_data)
df_accrued

,Description,Value
0,Days in coupon period (Feb 28 - Aug 31),184
1,Days accrued at t (Feb 28 - Apr 18),49
2,Accrued Interest at t (A_t),0.532609
3,Days accrued at T (Feb 28 - Aug 01),154
4,Accrued Interest at T (A_T),1.673913


### 1.2 Forward Price (No Interim Coupon)

We calculate the forward price for delivery at $T$. The spot purchase must be financed at the repo rate until delivery.

**Formula:**

$$
F(t, T) = (P_t + A_t)\left[1 + r_{\text{repo}} \cdot \tau_{\text{ACT/360}}\right] - A_T
$$

where:

- $P_t$: Clean spot price ($102\text{-}02$, or $102 + 2/32$)
- $r_{\text{repo}}$: $4.85\%$
- $\tau_{\text{ACT/360}}$: Day count fraction using Actual/360 convention

In [3]:
# Inputs
spot_price_32nds = 102 + 2/32
repo_rate = 0.0485

# Forward Price Calculation
# Dirty Price at t
dirty_price_t = spot_price_32nds + accrued_t

# Time fraction tau (Actual/360)
days_t_to_T = days_between(current_date, forward_date)
tau_act360 = days_t_to_T / 360.0

# Forward Price Formula
# Future Value of Dirty Price - Accrued Interest at T
fv_dirty_price = dirty_price_t * (1 + repo_rate * tau_act360)
forward_clean_price = fv_dirty_price - accrued_T

# Display the forward price data
forward_price_data = {
    "Description": [
        "Forward Price"
    ],
    "Value": [
        f"{forward_clean_price:.6f}"
    ]
}

df_forward_price = pd.DataFrame(forward_price_data)
df_forward_price

,Description,Value
0,Forward Price,102.372489


### 1.3 The Forward Drop

The **forward drop** is the difference between the spot price and the forward price:

$$
\text{Forward Drop} = P_t - F(t, T)
$$

**Explanation:**  
In our example, the forward price ($102.3725$) is higher than the spot price ($102.0625$). This results in a negative drop (also called a forward premium).

#### Carry Relationship

The financing cost (repo rate of 4.85%) significantly exceeds the bond's current yield accrual (4.00% coupon) over this period.

Mathematically:  
**Cost of Financing $>$ Coupon Income**

To prevent arbitrage, the forward price must be higher, compensating a short position for holding the asset while paying more in financing (repo) interest than is earned from the bond's coupon.

> The trade has **Negative Carry** (Income < Cost), implying a **Positive Cost of Carry** (Cost > Income) in the pricing formula, resulting in a forward price higher than the spot price.

In [7]:
# Forward Drop
forward_drop = spot_price_32nds - forward_clean_price

forward_drop_data = {
    "Description": [
        "Spot Price",
        "Forward Price",
        "Forward Drop (Spot - Forward)"
    ],
    "Value": [
        f"{spot_price_32nds:.6f}",
        f"{forward_clean_price:.6f}",
        f"{forward_drop:.6f}"
    ]
}

df_forward_drop = pd.DataFrame(forward_drop_data)
df_forward_drop

,Description,Value
0,Spot Price,102.062500
1,Forward Price,102.372489
2,Forward Drop (Spot - Forward),-0.309989


## 2. Carry Analysis